# ⚡ Build Faster AI Pipelines with Union Actors Example

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/Actors-Build-Faster-AI-Pipelines-with-Union/blob/main/faster_ai_pipelines_tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook contains examples of getting started with Union Actors.

Actors in Union let you reuse a container and its environment across multiple tasks, skipping the costly overhead of starting a fresh container every time.

If you're unfamiliar with Union, it is a workflow and inference platform, often used witn complex data pipelines and training AI models.

Read these for more information on Actors:
- Supercharge your AI workflows with Actors
- Union Actors documentation

In [ ]:
# install the union sdk
!pip install union

Sign up to [Union serverless](https://www.union.ai/) and run the code cell below. This will authenticate this environment to run on Union.

In [ ]:
!union create login --serverless --auth device-flow

## Simple Union Actors example

In [ ]:
%%writefile hello_actors.py

import union

actor = union.ActorEnvironment(
    name="my-actor-container", # unique name for the actor env
    replica_count=1,# Number of actor replicas to provision.
    ttl_seconds=120,  # Keep the actor alive even when idle
    requests=union.Resources(cpu="2", mem="300Mi"), # Compute resources actor requires
)
# update to just @task if you want to see the difference
@actor.task
def add_ints(num1: int, num2: int) -> int:
    return num1 + num2

@union.workflow
def wf_add()  -> int:
  num = add_ints(4, 2)
  num = add_ints(num, 5)
  num = add_ints(num, 2)
  num = add_ints(num, 4)

  return num

# once authenticated, run the workflow with:
# union run --remote hello_actors.py wf_add

In [ ]:
# run the workflows on Union
!union run --remote hello_actors.py wf_add

## Going Deeper: Caching with @actor_cache


!pip install transformers

In [ ]:
%%writefile actors_cache.py

import union
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from flytekit import ImageSpec, Resources
from union.actor import ActorEnvironment

image = ImageSpec(
    packages=[
        "union",
        "transformers",
        "torch",
        "accelerate",
    ],
    builder="union",
)

llm_actor = ActorEnvironment(
    name="gpu-llm-actor",
    container_image=image,
    replica_count=1,
    ttl_seconds=120,
    requests=Resources(cpu="1", mem="2000Mi", gpu="1"),
)


@union.actor_cache
def load_model(model_name: str = "microsoft/Phi-4-mini-instruct") -> pipeline:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cuda",
        torch_dtype="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return pipeline("text-generation", model=model, tokenizer=tokenizer)


@llm_actor.task
def actor_model_predict(query: str, prev_summary: str ="nothing yet.") -> str:
    nlp_pipeline = load_model()

    # Chain query using the previous output
    full_query = (
        f"{query} Previously you told me about {prev_summary}."
        "Keep answers short and 1 sentence long."
        "Include a list of all insects we discussed after the answer."
    )

    predictions = nlp_pipeline(full_query, batch_size=1, return_full_text=False)
    return predictions[0]["generated_text"]


@union.workflow
def wf_text_gen() -> str:
    result_ant = actor_model_predict(query="What is an ant?")
    result_bee = actor_model_predict(query="What is a bee", prev_summary=result_ant)
    result_wasp = actor_model_predict(query="What is a wasp", prev_summary=result_bee)
    return result_wasp

In [ ]:
!union run --remote actors_cache.py wf_text_gen